# Define input data

In [4]:
run = 504419 # low rate run, looks reasonable
# run = 504428 # high rate run, seems problematic
# datadir = f"/eos/user/t/tdietel/data/noise/{run}/"
datadir = f"/Users/tom/cernbox/data/noise/{run}/"

# Set up PyROOT environment and load digits file

In [2]:
import ROOT

%jsroot on

Welcome to JupyROOT 6.24/06


In [5]:
import pathlib

datadir = pathlib.Path(datadir)

digfile = ROOT.TFile(str(datadir / "trddigits.root"))
digtree = digfile.Get("o2sim")

# digtree.Print()

# Basic Checks

## Channel numbers
The channel numbers should be (almost) evenly distributed between 0 and 20. The pads at the edge of the chamber might be disabled, so it would be understanable if the outermost MCMs have the lowest or highest channels disabled.

For run 504428, the channel distribution looks very different. There seems to be something broken in generating the digits.

For run 504419, the channel distribution looks ok, but there are a few channel numbers greater than 20, and there are less digits from channels above 10 or so.

In [6]:
cnv = ROOT.TCanvas()
cnv.SetLogy()
digtree.Draw("mChannel")
cnv.Draw()

## Digits map
As the channels are somewhat weird, we should probably check if the digits are evenly distributed over chambers, ROBs and MCMs. Let's start with the chamber distribution. Some differences are expected, as parts of the detector are disabled, and the stack-2 chambers have only 6 ROBs instead of the 8 for chambers in stacks 0,1,3 and 4. 

In [ ]:
cnv_det = ROOT.TCanvas("cnv_det")
digtree.Draw("mDetector>>hdet(540,-0.5,539.5)")
cnv_det.Draw()

For 504428, that doesn't look right at all. Why would detector be between 0 and 31 - so it's not just the number within the supermodule. I'm not even sure if it makes sense to look at the MCM position within the chamber, but here we go.

For 504419, the distribution looks ok, although I would have expected an even more uniform distribution.

In [ ]:
cnv_map = ROOT.TCanvas("cnv_map")

digtree.Draw("mROB:mMCM>>hmap","1","colz")
# digtree.Draw("mROB:mMCM>>hmap","mDetector==4","colz")

# ROOT.hmap.Scale(1/54.) # I think I've seen 54 trigger records in this file
ROOT.hmap.Draw("text,same")
ROOT.hmap.SetStats(0)
cnv_map.Draw()

Again, the distribution is not as uniform as expected. Especially for 504428 that's strange, because the chambers were so different.